In [71]:
import pandas as pd
import optuna
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
data = pd.read_csv('../../data/learning_data.csv',index_col=None)

test_ratio = 0.2
    
X = data.drop('dengue', axis=1).values  
y = data['dengue'].values

# 分割インデックスの計算
split_index = int(len(data) * test_ratio)

# 前半20%をテストデータとして分割
X_test = X[:split_index]
X_train = X[split_index:]

y_test = y[:split_index]
y_train = y[split_index:]
    
#numpy配列をtensolに変換
"""X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)"""

'X_train_tensor = torch.tensor(X_train, dtype=torch.float32)\ny_train_tensor = torch.tensor(y_train, dtype=torch.float32)\nX_test_tensor = torch.tensor(X_test, dtype=torch.float32)\ny_test_tensor = torch.tensor(y_test, dtype=torch.float32)'

In [74]:
def objective(trial):
    # ハイパーパラメータの選択
    n_d = trial.suggest_int("n_d", 8, 64)
    n_a = trial.suggest_int("n_a", 8, 64)
    n_steps = trial.suggest_int("n_steps", 3, 10)
    gamma = trial.suggest_float("gamma", 1.0, 2.0)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-1)
    learning_rate = trial.suggest_float("learning_rate", 1e-7, 1e-1, log=True)
    
    # TabNetのモデルを作成
    model = TabNetClassifier(
        device_name=device,
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=learning_rate),
        mask_type="sparsemax"
    )

    # モデルの訓練
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_name=['test'],
        eval_metric=['accuracy'],
        max_epochs=1000,
        patience=10,
        batch_size=128,
        virtual_batch_size=128
    )

    # テストデータでの予測と評価
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [75]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# 最適化されたハイパーパラメータの取得
print("Best hyperparameters: ", study.best_params)
print("Best accuracy: ", study.best_value)

[I 2024-10-30 17:31:33,925] A new study created in memory with name: no-name-75804397-b43c-42da-837d-5f683f14a2e0
/home/gonken/anaconda3/envs/torch/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.81835 | test_accuracy: 0.48375 |  0:00:02s
epoch 1  | loss: 1.37225 | test_accuracy: 0.49227 |  0:00:04s
epoch 2  | loss: 1.12138 | test_accuracy: 0.4976  |  0:00:07s
epoch 3  | loss: 1.00193 | test_accuracy: 0.52158 |  0:00:09s
epoch 4  | loss: 0.92755 | test_accuracy: 0.5024  |  0:00:12s
epoch 5  | loss: 0.8622  | test_accuracy: 0.51731 |  0:00:14s
epoch 6  | loss: 0.84469 | test_accuracy: 0.50133 |  0:00:16s
epoch 7  | loss: 0.81245 | test_accuracy: 0.51199 |  0:00:18s
epoch 8  | loss: 0.78144 | test_accuracy: 0.51199 |  0:00:21s
epoch 9  | loss: 0.77822 | test_accuracy: 0.5008  |  0:00:23s
epoch 10 | loss: 0.76507 | test_accuracy: 0.50666 |  0:00:25s
epoch 11 | loss: 0.75912 | test_accuracy: 0.50932 |  0:00:28s
epoch 12 | loss: 0.75505 | test_accuracy: 0.504   |  0:00:30s
epoch 13 | loss: 0.74467 | test_accuracy: 0.50719 |  0:00:33s

Early stopping occurred at epoch 13 with best_epoch = 3 and best_test_accuracy = 0.52158


/home/gonken/anaconda3/envs/torch/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-30 17:32:07,911] Trial 0 finished with value: 0.5215769845498135 and parameters: {'n_d': 55, 'n_a': 23, 'n_steps': 5, 'gamma': 1.7112373553285747, 'lambda_sparse': 0.006231908646358611, 'learning_rate': 0.00011435500910348215}. Best is trial 0 with value: 0.5215769845498135.
/home/gonken/anaconda3/envs/torch/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.41352 | test_accuracy: 0.51678 |  0:00:02s
epoch 1  | loss: 1.44026 | test_accuracy: 0.52531 |  0:00:05s
epoch 2  | loss: 1.44685 | test_accuracy: 0.51252 |  0:00:08s
epoch 3  | loss: 1.44102 | test_accuracy: 0.50826 |  0:00:10s
epoch 4  | loss: 1.40332 | test_accuracy: 0.51572 |  0:00:13s
epoch 5  | loss: 1.42359 | test_accuracy: 0.51518 |  0:00:16s
epoch 6  | loss: 1.41253 | test_accuracy: 0.51518 |  0:00:18s
epoch 7  | loss: 1.40276 | test_accuracy: 0.51785 |  0:00:21s
epoch 8  | loss: 1.37674 | test_accuracy: 0.50932 |  0:00:24s
epoch 9  | loss: 1.39066 | test_accuracy: 0.51945 |  0:00:26s
epoch 10 | loss: 1.35181 | test_accuracy: 0.51518 |  0:00:28s
epoch 11 | loss: 1.37466 | test_accuracy: 0.52051 |  0:00:31s

Early stopping occurred at epoch 11 with best_epoch = 1 and best_test_accuracy = 0.52531


/home/gonken/anaconda3/envs/torch/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-30 17:32:40,788] Trial 1 finished with value: 0.5253063399041022 and parameters: {'n_d': 34, 'n_a': 20, 'n_steps': 7, 'gamma': 1.2036859755176745, 'lambda_sparse': 0.06923049262115193, 'learning_rate': 6.150986899828385e-06}. Best is trial 1 with value: 0.5253063399041022.
/home/gonken/anaconda3/envs/torch/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


...

epoch 0  | loss: 1.08267 | test_accuracy: 0.49281 |  0:00:02s
epoch 1  | loss: 0.8542  | test_accuracy: 0.50613 |  0:00:04s
epoch 2  | loss: 0.76212 | test_accuracy: 0.50932 |  0:00:07s
epoch 3  | loss: 0.7188  | test_accuracy: 0.49227 |  0:00:10s
epoch 4  | loss: 0.68906 | test_accuracy: 0.49494 |  0:00:13s
epoch 5  | loss: 0.6894  | test_accuracy: 0.49227 |  0:00:16s
epoch 6  | loss: 0.67958 | test_accuracy: 0.48269 |  0:00:18s
epoch 7  | loss: 0.67072 | test_accuracy: 0.504   |  0:00:21s
epoch 8  | loss: 0.66337 | test_accuracy: 0.49281 |  0:00:23s
epoch 9  | loss: 0.66777 | test_accuracy: 0.48375 |  0:00:26s
epoch 10 | loss: 0.66381 | test_accuracy: 0.51092 |  0:00:29s
epoch 11 | loss: 0.66324 | test_accuracy: 0.49867 |  0:00:32s
epoch 12 | loss: 0.66601 | test_accuracy: 0.496   |  0:00:34s
epoch 13 | loss: 0.66371 | test_accuracy: 0.504   |  0:00:37s
epoch 14 | loss: 0.65772 | test_accuracy: 0.51039 |  0:00:40s
epoch 15 | loss: 0.64956 | test_accuracy: 0.48961 |  0:00:43s
epoch 16

/home/gonken/anaconda3/envs/torch/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-30 20:20:13,983] Trial 198 finished with value: 0.51092168353756 and parameters: {'n_d': 51, 'n_a': 15, 'n_steps': 7, 'gamma': 1.6003395076333167, 'lambda_sparse': 0.04862388758614049, 'learning_rate': 0.00676151128177992}. Best is trial 4 with value: 0.5359616409163559.
/home/gonken/anaconda3/envs/torch/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.05292 | test_accuracy: 0.48961 |  0:00:02s
epoch 1  | loss: 0.83702 | test_accuracy: 0.51572 |  0:00:05s
epoch 2  | loss: 0.78947 | test_accuracy: 0.4992  |  0:00:08s
epoch 3  | loss: 0.74298 | test_accuracy: 0.5008  |  0:00:10s
epoch 4  | loss: 0.72183 | test_accuracy: 0.50773 |  0:00:13s
epoch 5  | loss: 0.70538 | test_accuracy: 0.50559 |  0:00:16s
epoch 6  | loss: 0.70225 | test_accuracy: 0.51145 |  0:00:18s
epoch 7  | loss: 0.68945 | test_accuracy: 0.48109 |  0:00:21s
epoch 8  | loss: 0.68257 | test_accuracy: 0.50506 |  0:00:24s
epoch 9  | loss: 0.66684 | test_accuracy: 0.48855 |  0:00:26s
epoch 10 | loss: 0.66182 | test_accuracy: 0.47682 |  0:00:29s
epoch 11 | loss: 0.65597 | test_accuracy: 0.49068 |  0:00:32s

Early stopping occurred at epoch 11 with best_epoch = 1 and best_test_accuracy = 0.51572


/home/gonken/anaconda3/envs/torch/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-30 20:20:47,633] Trial 199 finished with value: 0.515716568993074 and parameters: {'n_d': 49, 'n_a': 58, 'n_steps': 7, 'gamma': 1.5573098017869067, 'lambda_sparse': 0.017262700188333573, 'learning_rate': 0.003028088105971474}. Best is trial 4 with value: 0.5359616409163559.


Best hyperparameters:  {'n_d': 49, 'n_a': 17, 'n_steps': 5, 'gamma': 1.5791038792939252, 'lambda_sparse': 0.08222258804070917, 'learning_rate': 0.011539105483496466}
Best accuracy:  0.5359616409163559
